Introdução

Nesta última aula, iremos abordar o Topic Modeling, como último algoritmo não supervisionado para análise de dados textuais.Continuaremos com o mesmo conjunto de dados da BBC, para facilitar, porém com um estudo diferente sobre eles.


Fundamentação Teórica

Algoritmos de modelagem de tópicos são algoritmos frequentemente utilizados em processamento em grandes bases textuais à fim de obter informações, mostrando que um documento, diferente do cluster, pode pertencer a vários grupos de tópicos, sendo um tópico a distribuição de termos pertencentes à um único tema.(WANG & BLEI, 2011)

Spacy

Antes de iniciar a prática para implementar o código, vamos precisar instalar O Spacy, que é uma biblioteca para o Processamento de Linguagem Natural. Visto que vamos realizar o Topic Modeling, geralmente apenas os substantivos são necessários para gerar tópicos sobre uma base textual, sendo assim, com o spacy podemos realizar procedimentos sobre os textos utilizados a fim de filtrarmos somente os substantivos dos mesmos.

Para instalar o Spacy no Anaconda, basta seguir os passos abaixo:
 - Abra o Anaconda Prompt no modo Administrador
 - Com o prompt aberto, digite "conda install -c conda-forge spacy"
 - Após isso, digite "python -m spacy download en_core_web_sm"
 


Implementação do Código

Com o spacy instalado, podemos agora iniciar a implementação do código, vamos começar com a importação das bibliotecas inicialmente necessárias, que serão o Pandas para manipular o data frame e o Spacy para processar os textos.


In [13]:
import pandas as pd
import spacy

Com o Spacy importado, criaremos uma instância dele para podermos processar os textos do data frame

In [14]:
pln = spacy.load("en_core_web_sm", disable=['parser','ner'])

Com a instância do spacy criado, podemos agora importar os dados, alocando-os em um data frame, para que possamos realizar o processamento destes dados. Dessa forma, iremos pegar cada notícia e retirarmos as palavras desnecessárias, ou seja, no nosso caso, vamos deixar apenas os substantivos presentes em cada notícia.


In [15]:
df = pd.read_csv("noticias_bbc.csv")

def txt_substantivos(texto):
    resultado = []
    for doc in pln.pipe(texto):
        subst_texto= " ".join(token.lemma_ for token in doc if token.pos_=='NOUN')
        resultado.append(subst_texto)
    return resultado

df['texto']=txt_substantivos(df['texto'])

In [16]:
df.head()

,Unnamed: 0,texto,categoria
0,0,boss bag award executive business magazine tit...,0
1,1,copy bumper sale fi shooter game copy sale com...,4
2,2,msp climate warning climate change control dec...,2
3,3,pavey success view week race track bronze inju...,3
4,4,tory rethink association candidate election ag...,2


Com os dados devidamente processados, podemos seguir para o próximo passo, que é estruturá-los em uma BoW aliado ao TF-IDF.

In [17]:
#Importando TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vec= TfidfVectorizer(max_features=5000, stop_words="english", max_df=0.95,min_df=2)

features = vec.fit_transform(df['texto'])

Dado isso, vamos criar a instância do modelo de Topic Modeling, que é o NMF (Non-Negative Matrix Factorization), e treiná-lo com nossas features.

In [26]:
#Importando classificador NMF
from sklearn.decomposition import NMF
cls = NMF(n_components = 5, random_state=123)

cls.fit(features)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=123, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

Temos agora nosso modelo devidamente treinado, portanto, agora, precisamos extrair os resultados obtidos. Primeiramente vamos criar uma variável que contém o vetor com todas as 5000 palavras, para que possamos "pegá-las" através do índice no vetor mais tarde. Dessa forma, vamos criar a variável feature_names, como visto a seguir.


In [19]:
feature_names = vec.get_feature_names()

Como limitamos à 5000 mil palavras por vetor, vamos agora então limitar a quantidade de palavras que definem o nosso tópico, no nosso exemplo iremos fixar este valor em 15 palavras.

In [20]:
n_max_palavras = 15

Agora vamos à parte mais importante, que é de fato conseguir recuperar estas 15 palavras por tópico. Contudo vamos à uma breve explicação do código à seguir e das funções utilizadas nele.

Primeiramente, com enumerate() conseguimos literalmente enumerar um conjunto de dados. No nosso caso, iremos enumerar cls.components_, que é onde estão os vetores de palavras de cada tópico, sendo assim, através de i conseguimos acessar o número da enumeração dos vetores e através de vet_topico acessar os vetores que foram gerados para cada tópico.

Por fim, no segundo for, iremos aplicar o método argsort() em cada vetor dos tópicos. Dessa forma, com argsort() iremos reorganizar cada um desses vetores, fazendo com que o índices que contém  os menores valores (palavras menos significativas) se encontrem nos primeiros índices, enquanto que os índices das palavras que mais representam um tópico, ou seja, as mais significativas, sejam colocadas nos últimos índices. Portanto, aplicar o argsort iremos ordenar por ordem crescente de importâncias os índices dos termos de cada vetor.
Sendo assim, com o fancy indexing realizado em "[-1:-n_max_palavras-1:-1]", iremos pegar os últimos 15 itens do vetor, ou seja, iremos pegar os valores dos índices que iremos utilizar para recuperar as palavras em feature_names, e, por fim, imprimi-los para podermos analisar os resultados.


In [21]:
for i, vet_topico in enumerate(cls.components_):
    print(i, end=" ")
    for id_palavra in vet_topico.argsort()[-1:-n_max_palavras-1:-1]:
        print(feature_names[id_palavra], end=' ')
    print()

0 growth sale economy year company market share rate price firm profit oil analyst month quarter 
1 film award actor star actress director nomination movie year comedy role festival prize category ceremony 
2 game player match team injury club time win season coach goal victory title champion half 
3 election party government tax minister leader people campaign chancellor plan issue voter country taxis vote 
4 phone people music technology service user broadband software computer tv network device video site mobile 


Feito isso tudo, temos uma visão bastante significativa do resultado obtido, fazendo com que possamos aplicar novos dados para verificar em qual tópico eles se encaixam.

Sendo assim, vamos criar uma variável com algumas strings, simulando um vetor com notícias em cada índice, para que seja mostrado um resultado para novos dados.


In [24]:
novas_noticias = ["Oil prices have weakened sharply because of a combination of oversupply and a collapse in global demand due to the decline in economic activity caused by coronavirus lockdown measures.",
                  "The Haunting of Hill House premiered on the streamer back in 2018 to great success and critical acclaim. Last year, it was announced there would be a second season of the show, but with a completely different premise.",
                  "It might be that boxing takes longer to get going again than other sports because of the reliance on medical support. What struck me was the last statement by the board on recognising how NHS staff would need time to rest and recuperate."]

A seguir, iremos pegar este novo conjunto de dados e aplicá-los ao nosso classificador.

In [25]:
cls.transform(vec.transform(novas_noticias)).argsort(axis=1)[:,-1]

array([0, 2, 2], dtype=int64)

Com isso vemos que 2 dos 3 textos foram classificados corretamente de acordo com o nosso modelo.

REFERENCIAS

WANG, Chong; BLEI, David M. Collaborative topic modeling for recommending scientific articles. In: Proceedings of the 17th ACM SIGKDD international conference on Knowledge discovery and data mining. 2011. p. 448-456.